In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


# Se levanta el set de train generado

In [2]:
data_set = pd.read_csv("train_set_xgb.csv")
#X = data_set.loc[:, (data_set.columns != 'precio') & (data_set.columns != 'factor_pm')]
X = data_set.loc[:, (data_set.columns != 'precio')]
y = data_set.loc[:, ['precio']]

In [3]:
data_set.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'anio_publ'],
      dtype='object')

In [4]:
X.columns

Index(['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos',
       'anio_publ'],
      dtype='object')

In [5]:
y.columns

Index(['precio'], dtype='object')

# Se separa en train y en test

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [7]:
len(X_train)

168000

In [8]:
len(X_test)

72000

## Armamos el dataset sobre el cual cada XGBoost va a subsamplear con reposición.

### Esto se hace sobre el X_train con su respectivo label.

In [9]:
data_for_xgb = pd.concat([X_train, y_train], axis = 1)
data_for_xgb.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,anio_publ,precio
239101,0.0,3.0,2.0,3.0,196.0,160.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2014,1800000.0
96221,20.0,4.0,2.0,2.0,245.0,196.0,104734.0,NaN,NaN,0.0,0.0,0.0,1.0,0.0,2012,1500000.0
144453,5.0,3.0,1.0,2.0,108.0,120.0,58952.0,NaN,NaN,0.0,0.0,0.0,1.0,1.0,2015,850000.0
15620,10.0,3.0,1.0,3.0,200.0,147.0,49066.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2016,1550000.0
73157,NaN,NaN,0.0,NaN,NaN,250.0,24892.0,19.23339,-99.166992,0.0,0.0,0.0,0.0,0.0,2016,700000.0


# Se entrenan n XGB

Se define la cantidad de xgb a entrenar

In [10]:
n = 100

Se utilizan los valores encontrados en el Grid Search

In [1]:
xgb_created = []

In [11]:
for i in range(n):
    d_for_current_xgb = data_for_xgb.sample(frac = 1, replace = True, random_state = i)
    X_for_current_xgb = d_for_current_xgb.loc[:, (d_for_current_xgb.columns != 'precio')]
    y_for_current_xgb = d_for_current_xgb.loc[:, ['precio']]
    current_xgb = xgb.XGBRegressor(objective ='reg:squarederror', 
                                colsample_bytree = 0.8, 
                                gamma = 0.5,
                                 learning_rate = 0.1,
                                 max_depth = 5,
                                 min_child_weight = 10,
                                 subsample = 1.0, n_estimators = 2400, n_jobs = -1, random_state = i)
    current_xgb.fit(X_for_current_xgb, y_for_current_xgb)
    current_xgb.save_model("xgb_{}".format(i))
    xgb_created.append(current_xgb)
    print("XGB number {} created".format(i))

XGB number 0 created
XGB number 1 created
XGB number 2 created
XGB number 3 created
XGB number 4 created
XGB number 5 created
XGB number 6 created
XGB number 7 created
XGB number 8 created
XGB number 9 created
XGB number 10 created
XGB number 11 created
XGB number 12 created
XGB number 13 created
XGB number 14 created
XGB number 15 created
XGB number 16 created
XGB number 17 created
XGB number 18 created
XGB number 19 created
XGB number 20 created
XGB number 21 created
XGB number 22 created
XGB number 23 created
XGB number 24 created
XGB number 25 created
XGB number 26 created
XGB number 27 created
XGB number 28 created
XGB number 29 created
XGB number 30 created
XGB number 31 created
XGB number 32 created
XGB number 33 created
XGB number 34 created
XGB number 35 created
XGB number 36 created
XGB number 37 created
XGB number 38 created
XGB number 39 created
XGB number 40 created
XGB number 41 created
XGB number 42 created
XGB number 43 created
XGB number 44 created
XGB number 45 create

#### Se usa un subsample de igual tamaño que X_train

### Disclaimer: No se testea el armado de cada XGBoost porque asumo que funciona debido a que es el mismo modelo con los mismos hiperparametros que se usó anteriormente y lo único que cambia es que se está entrenando sobre un subsample de los datos originales.

# Se arma el DF de precios para entrenar el RF

In [19]:
RF_train = pd.DataFrame(index = X_train.index.copy())

In [20]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_train)
    RF_train['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
RF_train.head()

xgb_0 finished pred
xgb_1 finished pred
xgb_2 finished pred
xgb_3 finished pred
xgb_4 finished pred
xgb_5 finished pred
xgb_6 finished pred
xgb_7 finished pred
xgb_8 finished pred
xgb_9 finished pred
xgb_10 finished pred
xgb_11 finished pred
xgb_12 finished pred
xgb_13 finished pred
xgb_14 finished pred
xgb_15 finished pred
xgb_16 finished pred
xgb_17 finished pred
xgb_18 finished pred
xgb_19 finished pred
xgb_20 finished pred
xgb_21 finished pred
xgb_22 finished pred
xgb_23 finished pred
xgb_24 finished pred
xgb_25 finished pred
xgb_26 finished pred
xgb_27 finished pred
xgb_28 finished pred
xgb_29 finished pred
xgb_30 finished pred
xgb_31 finished pred
xgb_32 finished pred
xgb_33 finished pred
xgb_34 finished pred
xgb_35 finished pred
xgb_36 finished pred
xgb_37 finished pred
xgb_38 finished pred
xgb_39 finished pred
xgb_40 finished pred
xgb_41 finished pred
xgb_42 finished pred
xgb_43 finished pred
xgb_44 finished pred
xgb_45 finished pred
xgb_46 finished pred
xgb_47 finished pred
xg

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,1731590.375,1.687190e+06,1708003.750,1.777730e+06,1611353.250,1685426.375,1805369.375,1.737057e+06,1.724486e+06,1.715475e+06,...,1.768250e+06,1.722863e+06,1762664.000,1.768310e+06,1.819990e+06,1853000.125,1861383.500,1.779566e+06,1.795215e+06,1776192.000
1,1861900.750,1.811220e+06,1833665.875,1.901733e+06,1891049.375,1797504.375,1612934.125,1.669204e+06,1.775289e+06,1.768255e+06,...,1.454514e+06,1.805304e+06,1809681.125,1.840597e+06,1.724257e+06,1820837.125,1700720.500,1.613766e+06,1.632273e+06,1756345.500
2,849920.375,9.141216e+05,764238.875,7.420772e+05,914536.625,917239.875,820651.000,9.734854e+05,7.981991e+05,9.497426e+05,...,8.563761e+05,9.187719e+05,1052048.250,9.042729e+05,8.909278e+05,820594.375,806802.500,8.459802e+05,7.979854e+05,869928.750
3,1854745.125,1.846927e+06,1866113.750,2.002066e+06,1790563.375,1752173.500,1905055.250,1.927715e+06,1.755575e+06,1.823570e+06,...,1.733450e+06,1.873017e+06,1970537.250,1.770525e+06,1.734895e+06,1896902.625,1909469.500,1.757820e+06,1.954052e+06,2038684.125
4,1172485.375,1.489644e+06,1405968.000,1.665526e+06,1913289.000,2042179.625,1899234.125,1.338867e+06,1.773599e+06,2.308966e+06,...,1.928238e+06,1.012067e+06,1715285.500,2.078893e+06,1.590580e+06,1685367.000,1949662.625,1.152652e+06,2.274063e+06,1052914.375


In [21]:
len(RF_train)

168000

In [22]:
len(y_train)

168000

## Exporto el RF_train

In [23]:
RF_train.to_csv("RF_train", index = False)

## Armo el DF para RF a partir de X_train

In [25]:
RF_test = pd.DataFrame(index = X_test.index.copy())


In [26]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_test)
    RF_test['pred_{}'.format(i)] = current_price_pred
RF_test.head()

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_90,pred_91,pred_92,pred_93,pred_94,pred_95,pred_96,pred_97,pred_98,pred_99
0,1.798102e+06,1446248.625,1407567.250,1.653867e+06,1.402460e+06,1442624.500,1.703511e+06,1.649957e+06,1.435246e+06,2.082397e+06,...,1.516377e+06,1.462098e+06,1666077.750,1406809.000,1636870.50,1.727082e+06,1.349754e+06,1.649911e+06,1.931766e+06,1.392859e+06
1,2.560048e+06,2756970.500,2667663.500,2.664034e+06,2.613616e+06,2601715.500,2.486792e+06,2.557274e+06,2.464232e+06,2.712020e+06,...,2.657490e+06,2.554301e+06,2572289.500,2481868.250,2371856.00,2.742639e+06,2.583326e+06,2.616145e+06,2.509907e+06,2.596720e+06
2,2.664327e+06,2246674.750,2728572.750,2.303223e+06,2.519035e+06,2831999.250,2.267160e+06,2.268172e+06,2.876760e+06,2.509534e+06,...,2.332474e+06,1.777443e+06,1496987.875,1867642.875,2871201.75,2.292293e+06,2.148274e+06,1.874294e+06,2.647240e+06,2.183096e+06
3,1.309244e+06,1342845.750,1244864.500,1.376778e+06,1.282279e+06,1426242.000,1.322379e+06,1.186508e+06,1.312307e+06,1.371620e+06,...,1.146656e+06,1.328758e+06,1468304.625,1385828.250,1291130.25,1.421451e+06,1.516852e+06,1.931046e+06,1.090346e+06,1.232119e+06
4,2.889435e+05,277751.500,327708.875,3.096952e+05,3.009607e+05,205150.625,2.084183e+05,2.095406e+05,3.348800e+05,2.344241e+05,...,3.102430e+05,1.753795e+05,380991.875,250429.625,274002.75,2.639580e+05,2.213890e+05,3.714423e+05,2.726218e+05,2.091287e+05


# Se exporta RF_test

In [2]:
RF_test.to_csv("RF_test", index = False)

NameError: name 'RF_test' is not defined